In [201]:
import pandas as pd
import os
import numpy as np
from sklearn.feature_selection import chi2
import numpy as np
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import SelectKBest
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression

In [190]:
!pip install mlxtend

In [3]:
current_path = os.getcwd()
dirname, filename = os.path.split(current_path)
print(dirname, filename)

C:\Users\saksh\Documents\20220505_CouponAcceptance_with_MLFlow notebooks


In [4]:
train_fengg_file_path = os.path.join(dirname, "traindf_with_feature_engg.csv")
traindf_with_feature_engg= pd.read_csv(train_fengg_file_path)

In [5]:
traindf_with_feature_engg.head()

,destination,passanger,weather,temperature,time,coupon,expiration,gender,age,maritalStatus,...,CoffeeHouse,CarryAway,RestaurantLessThan20,Restaurant20To50,toCoupon_GEQ5min,toCoupon_GEQ15min,toCoupon_GEQ25min,direction_same,direction_opp,Y
0,2,3,2,55,2,2,2,1,2,2,...,1,2,3,3,1,0,0,0,1,1
1,2,1,2,80,2,2,1,1,2,2,...,1,2,3,3,1,1,0,0,1,1
2,2,3,2,55,2,2,2,1,2,2,...,1,2,3,3,1,1,0,0,1,1
3,1,3,2,55,2,1,2,1,2,2,...,1,2,3,3,1,0,0,1,0,1
4,1,3,2,55,1,1,2,1,2,2,...,1,2,3,3,1,1,1,0,1,0


## Analysis Functions

In [8]:
def spearman_corr(df, dep_col):
    spear_coef = df.corr(method="spearman")[dep_col]
    return spear_coef

In [35]:
def convert_result_series_to_df(res, colname_list):
    res_df=pd.DataFrame(res)
    res_df.columns = colname_list
    return res_df

In [112]:
def chisquare_test(df, dep_col):
    list_chi_score = []
    list_col_names = []
    for i in df.columns:
        X = np.array(df[i]).reshape(-1,1)
        y = df[dep_col]
        chi_scores = chi2(X,y)
        list_chi_score.append(chi_scores[1][0])
        chi_score_series = pd.Series( v for v in list_chi_score )
        list_col_names.append(i)
        chi_score_series.index = list_col_names
        chi_score_series.sort_values(ascending=False)
    return chi_score_series

In [37]:
def convert_list_to_series(lst):
    series = pd.Series( v for v in lst )
    return series

In [65]:
def col_list_with_1_class(df, num_list):
    col_list_to_drop = []
    for i in num_list:
        if len(df[i].value_counts()) ==1:
            col_list_to_drop.append(i)
    return col_list_to_drop

In [66]:
def Categorical_Numerical_Features_split(df):
    categorical_data = df.select_dtypes(exclude=[np.number])
    cat_list = list(categorical_data.columns)
    numeric_data = df.select_dtypes(include=[np.number])
    num_list = list(numeric_data.columns)
    return cat_list, num_list

In [72]:
def mutualinfo_values(xtrain, ytrain):
    mutual_info = mutual_info_classif(xtrain, ytrain)
    mutual_info = pd.Series(mutual_info)
    mutual_info.index = xtrain.columns
    mutual_info.sort_values(ascending=False)
    return mutual_info

In [110]:
def woe_iv_values(xtrain, ytrain):
    woe_iv_sum_list =[]
    col_list= []
    for i in xtrain.columns:
        df_woe_iv = (pd.crosstab(xtrain[i],ytrain,
                        normalize='columns')
                .assign(woe=lambda dfx: np.log(dfx[1] / dfx[0]))
                .assign(iv=lambda dfx: np.sum(dfx['woe']*
                                            (dfx[1]-dfx[0]))))
        #print(df_woe_iv)
        woe_iv_sum_list.append(df_woe_iv.iv.sum())
        woe_iv_sum_series = pd.Series( v for v in woe_iv_sum_list )
        col_list.append(i)
        woe_iv_sum_series.index = col_list
        woe_iv_sum_series.sort_values(ascending=False)
    return woe_iv_sum_series

In [178]:
def Analysis_Report(list_of_df, key):
    merged_df = pd.concat((list_of_df), axis=1)
    return merged_df

In [186]:
def selectkbest_features_mutual_classif(xtrain, ytrain, K):
    sel_five_cols = SelectKBest(mutual_info_classif, k=K)
    sel_five_cols.fit(xtrain, ytrain)
    five_best_features = list(xtrain.columns[sel_five_cols.get_support()])
    return five_best_features
    

In [206]:
def modelling_data_linReg_n_Feature_selection(xtrain, ytrain, k_features, forward):
    ##Finding Best features using Backward Algorithm with Linear Regression 
    log_reg = LogisticRegression()
    sfs_logReg = sfs(log_reg, k_features=10, forward=forward, verbose=2, scoring='neg_mean_squared_error')
    sfs_logReg = sfs_logReg.fit(xtrain, ytrain)
    feat_names = list(sfs_logReg.k_feature_names_)
    return feat_names

In [205]:
def modelling_data_logReg_n_Feature_selection(xtrain, ytrain, k_features, forward):
    ##Finding Best features using Backward Algorithm with Linear Regression 
    lin_reg = LinearRegression()
    sfs_linReg = sfs(lin_reg, k_features=10, forward=forward, verbose=2, scoring='neg_mean_squared_error')
    sfs_linReg = sfs_linReg.fit(xtrain, ytrain)
    feat_names = list(sfs_linReg.k_feature_names_)
    return feat_names

## Action Functions

In [29]:
def splitdf_into_Xtrain_n_Ytrain(df, dep_col):
    Ytrain = df[dep_col]
    Xtrain = df.drop([dep_col], axis=1)
    return Xtrain, Ytrain

## Flow

In [182]:
res_spear_coef = spearman_corr(traindf_with_feature_engg, 'Y')
spear_coef_df = convert_result_to_df(res_spear_coef, colname_list=['spearman_coefficient'] )

In [183]:
lst_score_series= chisquare_test(traindf_with_feature_engg, 'Y')
chi_square_score_df= convert_result_series_to_df(lst_score_series, ['chi_square_scores'])

In [67]:
categorical_list, numerical_list = Categorical_Numerical_Features_split(traindf_with_feature_engg)

In [69]:
DROP_COL_LIST_WITH_ONE_CLASS = col_list_with_1_class(traindf_with_feature_engg, numerical_list)
print(DROP_COL_LIST_WITH_ONE_CLASS)

['toCoupon_GEQ5min']


In [75]:
Xtrain, Ytrain = splitdf_into_Xtrain_n_Ytrain(traindf_with_feature_engg, 'Y')

In [179]:
mutual_info_series = mutualinfo_values(Xtrain, Ytrain)
mutual_info_df = convert_result_series_to_df(mutual_info_series, ['mutual_info_values'])

In [180]:
weight_of_evidence_Info_series = woe_iv_values(Xtrain, Ytrain)
WOE_IV_Sum_df = convert_result_series_to_df(weight_of_evidence_Info_series, ['WOE_IV_values'])

In [177]:
dfs_list = [spear_coef_df[:-1],chi_square_score_df[:-1], mutual_info_df, WOE_IV_Sum_df]
key = mutual_info_df.index
Analysis_Report(dfs_list, key)

,spearman_coefficient,chi_square_scores,mutual_info_values,WOE_IV_values
destination,0.148306,1.809158e-04,0.014473,0.180716
passanger,-0.155293,9.810007e-08,0.009542,0.305876
weather,0.106340,4.316109e-02,0.002489,0.091316
temperature,0.074146,6.264668e-28,0.000000,0.067988
time,0.142619,6.158616e-04,0.013169,0.165826
coupon,0.173798,2.491824e-04,0.027598,0.245306
expiration,0.121140,3.024467e-03,0.000000,0.119703
gender,0.024922,5.291109e-01,0.000000,0.005061
age,0.084311,3.185051e-02,0.001472,0.058170
maritalStatus,0.054095,1.831532e-01,0.000000,0.023825


In [187]:
TEN_BEST_FEATURES_WITH_MUTUAL_INFO_CLASSIF = selectkbest_features_mutual_classif(Xtrain, Ytrain, 10)
TEN_BEST_FEATURES_WITH_MUTUAL_INFO_CLASSIF

['destination',
 'passanger',
 'time',
 'coupon',
 'expiration',
 'age',
 'occupation',
 'CoffeeHouse',
 'Restaurant20To50',
 'toCoupon_GEQ25min']

In [199]:
TEN_BEST_FEATURES_WITH_LIN_REG_BKWD = modelling_data_linReg_n_Feature_selection(Xtrain, Ytrain,10,  False)
TEN_BEST_FEATURES_WITH_LIN_REG_BKWD

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed:    0.4s finished

[2022-05-06 20:51:08] Features: 23/10 -- score: -0.21662643638994053[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  23 out of  23 | elapsed:    0.3s finished

[2022-05-06 20:51:09] Features: 22/10 -- score: -0.2162939886754637[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  22 out of  22 | elapsed:    0.4s finished

[2022-05-06 20:51:09] Features: 21/10 -- score: -0.21601909517856105[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]

['destination',
 'passanger',
 'weather',
 'coupon',
 'expiration',
 'age',
 'education',
 'CoffeeHouse',
 'Restaurant20To50',
 'direction_same']

In [200]:
TEN_BEST_FEATURES_WITH_LIN_REG_FWD = modelling_data_linReg_n_Feature_selection(Xtrain, Ytrain,10,  True)
TEN_BEST_FEATURES_WITH_LIN_REG_FWD

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed:    0.1s finished

[2022-05-06 20:51:28] Features: 1/10 -- score: -0.23856474625707697[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  23 out of  23 | elapsed:    0.1s finished

[2022-05-06 20:51:28] Features: 2/10 -- score: -0.23274223747664885[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  22 out of  22 | elapsed:    0.1s finished

[2022-05-06 20:51:28] Features: 3/10 -- score: -0.2272932220498587[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: D

['passanger',
 'weather',
 'time',
 'coupon',
 'expiration',
 'age',
 'education',
 'CoffeeHouse',
 'Restaurant20To50',
 'direction_same']

In [207]:
TEN_BEST_FEATURES_WITH_LOG_REG_BKWD = modelling_data_logReg_n_Feature_selection(Xtrain, Ytrain,10,  False)
TEN_BEST_FEATURES_WITH_LOG_REG_BKWD

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed:    0.4s finished

[2022-05-06 20:56:46] Features: 23/10 -- score: -0.21662643638994053[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  23 out of  23 | elapsed:    0.3s finished

[2022-05-06 20:56:47] Features: 22/10 -- score: -0.2162939886754637[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  22 out of  22 | elapsed:    0.3s finished

[2022-05-06 20:56:47] Features: 21/10 -- score: -0.21601909517856105[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]

['destination',
 'passanger',
 'weather',
 'coupon',
 'expiration',
 'age',
 'education',
 'CoffeeHouse',
 'Restaurant20To50',
 'direction_same']

In [208]:
TEN_BEST_FEATURES_WITH_LOG_REG_FWD = modelling_data_logReg_n_Feature_selection(Xtrain, Ytrain,10,  True)
TEN_BEST_FEATURES_WITH_LOG_REG_FWD

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed:    0.1s finished

[2022-05-06 20:57:55] Features: 1/10 -- score: -0.23856474625707697[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  23 out of  23 | elapsed:    0.2s finished

[2022-05-06 20:57:55] Features: 2/10 -- score: -0.23274223747664885[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  22 out of  22 | elapsed:    0.1s finished

[2022-05-06 20:57:55] Features: 3/10 -- score: -0.2272932220498587[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: D

['passanger',
 'weather',
 'time',
 'coupon',
 'expiration',
 'age',
 'education',
 'CoffeeHouse',
 'Restaurant20To50',
 'direction_same']

## Remarks

### CONFIG

In [ ]:
DROP_COL_LIST_WITH_ONE_CLASS = ['toCoupon_GEQ5min']
TEN_BEST_FEATURES_WITH_MUTUAL_INFO_CLASSIF = ['destination','passanger','weather','time','coupon', 'expiration', 
                                              'education','CoffeeHouse','toCoupon_GEQ15min','direction_same']
TEN_BEST_FEATURES_WITH_LIN_REG_BKWD = ['destination','passanger','weather','coupon','expiration','age','education','CoffeeHouse',
                                  'Restaurant20To50','direction_same']
TEN_BEST_FEATURES_WITH_LIN_REG_FWD = ['passanger','weather','time','coupon','expiration','age','education',
                                      'CoffeeHouse','Restaurant20To50','direction_same']
TEN_BEST_FEATURES_WITH_LOG_REG_BKWD = ['destination','passanger','weather','coupon','expiration','age','education',
                                       'CoffeeHouse','Restaurant20To50','direction_same']
TEN_BEST_FEATURES_WITH_LOG_REG_FWD =['passanger','weather','time','coupon','expiration','age','education','CoffeeHouse',
                                     'Restaurant20To50','direction_same']